**시작**


In [9]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import splitfolders
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from tqdm.notebook import tqdm
import pickle
import platform
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [6]:
platform.platform()

'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic'

In [7]:
!cat /proc/meminfo

MemTotal:       36952008 kB
MemFree:        31366484 kB
MemAvailable:   35301768 kB
Buffers:          352412 kB
Cached:          3838240 kB
SwapCached:            0 kB
Active:           600284 kB
Inactive:        4410336 kB
Active(anon):        860 kB
Inactive(anon):   747268 kB
Active(file):     599424 kB
Inactive(file):  3663068 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              3844 kB
Writeback:             0 kB
AnonPages:        820104 kB
Mapped:           506080 kB
Shmem:              1192 kB
KReclaimable:     133712 kB
Slab:             235716 kB
SReclaimable:     133712 kB
SUnreclaim:       102004 kB
KernelStack:       12768 kB
PageTables:        14848 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    18476004 kB
Committed_AS:    5584796 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       16644 kB
VmallocChunk:          0 kB
Percpu:          

In [7]:
folder_path = '/content/drive/MyDrive/Training'

In [8]:
label_names = os.listdir(folder_path)

In [9]:
label_names

['0plus', '1plus', '2plus']

In [10]:
#묶여서 하나의 리스트에 넣어야 우리가 핸들링 하기 편하다.
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'0plus': ['/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005606.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005613.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005605.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005608.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005611.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005604.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005612.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005617.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005619.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005616.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005634.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005655.jpg',
  '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005

In [11]:
dataset

SyntaxError: ignored

In [11]:
!mkdir resized # resize 디렉토리
!mkdir resized/0plus
!mkdir resized/1plus
!mkdir resized/2plus

In [ ]:
import shutil
shutil.rmtree('/content/resized')#  디렉토리삭제

In [12]:
dataset.items() #딕셔너리 키값 쌍 얻기, 반복적인 구조에 좋다

dict_items([('0plus', ['/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005606.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005613.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005605.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005608.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005611.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005604.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005612.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005617.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005619.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005616.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005634.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005655.jpg', '/content/drive/MyDrive/Training/0plus/QC_pig_segmentation_1+_005651.jpg', '/

In [13]:
#직사각형 이미지를 불러드려서 긴쪽 짧은쪽 맞추고, 정사각형을 만든다(패딩을 씌운다.) 양쪽에 블랙패딩

for label, filenames in tqdm(dataset.items()):
    for filename in tqdm(filenames):
        img = cv2.imread(filename)

        # 이미지의 x, y가 224이 넘을 경우 작게해주기
        percent = 1
        if(img.shape[1] > img.shape[0]) :       # 이미지의 가로가 세보다 크면 가로를 640으로 맞추고 세로를 비율에 맞춰서
            percent = 224/img.shape[1]          #열이크다면 224로 나눠줘라
        else :
            percent = 224/img.shape[0]          #행이크다면 224로 나눠줘라

        img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
                # 이미지 범위 지정
        y,x,h,w = (0,0,img.shape[0], img.shape[1])

        # 그림 주변에 검은색으로 칠하기
        w_x = (224-(w-x))/2  # w_x = (224 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
        h_y = (224-(h-y))/2

        if(w_x < 0):         # 크기가 -면 0으로 지정
            w_x = 0
        elif(h_y < 0):
            h_y = 0

        M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
        img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  
       
        # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
        cv2.imwrite('/content/resized/{0}/{1}'.format(label, filename.split("/")[-1]) , img_re)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3029 [00:00<?, ?it/s]

  0%|          | 0/3076 [00:00<?, ?it/s]

  0%|          | 0/2476 [00:00<?, ?it/s]

어그먼테이션


In [14]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2, #넓이 이동
    height_shift_range=0.2,
    shear_range=0.2, # 사다리꼴 우측상단,
    zoom_range=0.2, # 줌땅기는거
    horizontal_flip=True, #뒤집는거
    vertical_flip=True,
)

In [20]:
folder_path = '/content/resized'

In [21]:
for label in  tqdm(os.listdir(folder_path)):
    label_path = folder_path + '/' + label + '/'
    for filename in tqdm(os.listdir(label_path)): 
        filepath = label_path + filename

        img = load_img(filepath)
        # img 출력
        # plt.imshow(img)
        # break
        x = img_to_array(img)
        # x.shape 출력
        # print(x.shape)
        # break
        x = x.reshape((1,) + x.shape)

        i = 0
        # flow : augmentation 함수 
        #눈으로 직접볼수 있음 저장이 된다.(나중엔 다른걸 씀 why? 용량문제)
        for batch in datagen.flow(x, batch_size=1, #포문에 갑자기 함수가? generate 란 친구다. 한번실행시키고 종속코드가 
                                save_to_dir=label_path, save_prefix=label, save_format='jpg'):
            i += 1
            if i > 1:
                break  #flow 때문에 break

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3029 [00:00<?, ?it/s]

  0%|          | 0/2476 [00:00<?, ?it/s]

  0%|          | 0/3076 [00:00<?, ?it/s]

In [22]:
folder_path = '/content/resized'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'0plus': ['/content/resized/0plus/0plus_0_2142.jpg',
  '/content/resized/0plus/QC_pig_segmentation_1+_002268.jpg',
  '/content/resized/0plus/QC_pig_segmentation_1+_001216.jpg',
  '/content/resized/0plus/0plus_0_163.jpg',
  '/content/resized/0plus/0plus_0_9148.jpg',
  '/content/resized/0plus/0plus_0_8774.jpg',
  '/content/resized/0plus/0plus_0_9353.jpg',
  '/content/resized/0plus/0plus_0_7717.jpg',
  '/content/resized/0plus/QC_pig_segmentation_1+_001238.jpg',
  '/content/resized/0plus/QC_pig_segmentation_1+_006084.jpg',
  '/content/resized/0plus/0plus_0_5173.jpg',
  '/content/resized/0plus/0plus_0_7697.jpg',
  '/content/resized/0plus/0plus_0_873.jpg',
  '/content/resized/0plus/0plus_0_5854.jpg',
  '/content/resized/0plus/0plus_0_50.jpg',
  '/content/resized/0plus/QC_pig_segmentation_1+_007642.jpg',
  '/content/resized/0plus/0plus_0_9237.jpg',
  '/content/resized/0plus/QC_pig_segmentation_1+_005890.jpg',
  '/content/resized/0plus/0plus_0_4055.jpg',
  '/content/resized/0plus/QC_pig_segme

In [23]:
dataset.items() 

dict_items([('0plus', ['/content/resized/0plus/0plus_0_2142.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_002268.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_001216.jpg', '/content/resized/0plus/0plus_0_163.jpg', '/content/resized/0plus/0plus_0_9148.jpg', '/content/resized/0plus/0plus_0_8774.jpg', '/content/resized/0plus/0plus_0_9353.jpg', '/content/resized/0plus/0plus_0_7717.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_001238.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_006084.jpg', '/content/resized/0plus/0plus_0_5173.jpg', '/content/resized/0plus/0plus_0_7697.jpg', '/content/resized/0plus/0plus_0_873.jpg', '/content/resized/0plus/0plus_0_5854.jpg', '/content/resized/0plus/0plus_0_50.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_007642.jpg', '/content/resized/0plus/0plus_0_9237.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_005890.jpg', '/content/resized/0plus/0plus_0_4055.jpg', '/content/resized/0plus/QC_pig_segmentation_1+_000464.jpg', '/

In [ ]:
#데이터셋 나누기 (color)
splitfolders.ratio('resized', output='dataset', seed=77, ratio=(0.9, 0.1)) 

Copying files: 9596 files [00:01, 6846.64 files/s]


In [ ]:
!mkdir gs
!mkdir gs/0plus
!mkdir gs/1plus
!mkdir gs/2plus

In [ ]:
!mv re

In [ ]:
#그레이스케일
import cv2
import os
import numpy as np
from PIL import Image
 
path = '/content/resized'
for label in os.listdir(path):
    sub_path = path+'/'+label+'/'
    for filename in os.listdir(sub_path):
      
        img = Image.open(sub_path+filename).convert('L')
        img_numpy = np.array(img, 'uint8')
        cv2.imwrite('/content/gs/{0}/{1}'.format(label, filename.split("/")[-1]) , img_numpy)
        # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장


In [ ]:
#데이터셋 나누기
splitfolders.ratio('gs', output='dataset', seed=77, ratio=(0.9, 0.1)) 

Copying files: 9596 files [00:01, 7089.08 files/s]


In [ ]:
#train data set 나누기
folder_path = '/content/dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'2plus': ['/content/dataset/train/2plus/QC_pig_segmentation_2_008609.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_004712.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_005217.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_002546.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_004133.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_005170.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_000687.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_001858.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_009485.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_003183.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_003318.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_007046.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_003799.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2_004124.jpg',
  '/content/dataset/train/2plus/QC_pig_segmentation_2

In [24]:
label2index = {'0plus' : 0, '1plus': 1, '2plus':2}
#딕셔너리를 만들어 , 해당 키 값의 해당하는 친구를 라벨을 숫자로 바꿔줄수있겠다.

In [25]:
x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_train.append(image)
        y_train.append(label2index[label]) # label을 index로 변경

In [26]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = x_train.astype('int8')
x_train.shape, y_train.shape

((21534, 224, 224, 3), (21534,))

Load validation, data, test data

In [ ]:
#Validation data 
folder_path = '/content/dataset/'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

In [ ]:
x_val, y_val = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_val.append(image)
        y_val.append(label2index[label]) # label을 index로 변경
x_val, y_val= np.array(x_val), np.array(y_val)
x_val = x_val.astype('int8')
x_val.shape, y_val.shape

((851, 224, 224, 3), (851,))

test set

In [16]:
#Test dataset 
folder_path = '/content/teresized'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'2plus': ['/content/teresized/2plus/QC_pig_segmentation_2_008607.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_008689.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009564.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009438.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009352.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_008772.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009554.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009043.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009504.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_008827.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009207.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009084.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_008851.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_008837.jpg',
  '/content/teresized/2plus/QC_pig_segmentation_2_009106.jpg',
  '/content/teresized/2plus/QC_pig_segmentatio

In [19]:
x_test, y_test = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename) # img를 array 형태로 변경

        x_test.append(image)
        y_test.append(label2index[label]) # label을 index로 변경

x_test, y_test = np.array(x_test), np.array(y_test)
x_test = x_test.astype('int8')
x_test.shape, y_test.shape

((1065, 224, 224, 3), (1065,))

In [40]:
#데이터 갯수, 차원, 채널 확인
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((29001, 224, 224, 3), (29001,), (1065, 224, 224, 3), (1065,))

In [27]:
# Zero Centering

def zero_mean(image):
    # zero-centering
    return np.mean(image, axis=0) #axis(중심선) 를  0으로 두고 평균을 낸다.

In [28]:
zero_mean_img = zero_mean(x_train).astype('int8') 

In [43]:
zero_mean_img.shape 

(224, 224, 3)

In [29]:
x_train -= zero_mean_img

In [45]:
x_test -= zero_mean_img

In [30]:
x_train.shape,y_train.shape, x_test.shape, y_test.shape

NameError: ignored

In [31]:
#데이터저장 (array, 제로센터링, dataset을 나눈 데이터를 피클 파일에 저장)
#why? resize와 같은 시간이 오래걸리는 작업을 계속 반복을 해야함 런타임이 끊어져서 그래서 파일크기가 적은 피클로 저장한다.
import pickle
folder_path='/content/drive'
with open(folder_path+'xtrain.pickle', 'wb') as f:
    pickle.dump(x_train, f, protocol=4)
with open(folder_path+'ytrain.pickle', 'wb') as f:
    pickle.dump(y_train, f)
#with open(folder_path+'1xtest.pickle', 'wb') as f:
    # pickle.dump(x_test, f)
#with open(folder_path+'1ytest.pickle', 'wb') as f:
   # pickle.dump(y_test, f)

In [32]:
!mv drivextrain.pickle /content/drive/MyDrive
!mv driveytrain.pickle /content/drive/MyDrive
#!mv drivetestx.pickle /content/drive/MyDrive
#!mv drivetesty.pickle /content/drive/MyDrive

In [ ]:
!mv --help


In [ ]:
!mv -R* resized /content/dirve/MyDrive

mv: invalid option -- 'R'
Try 'mv --help' for more information.


In [11]:
#데이터불러오기
with open('/content/drive/MyDrive/picklefile/trainx.pickle','rb') as f:
    x_train= pickle.load(f)
with open('/content/drive/MyDrive/picklefile/trainy.pickle','rb') as f:
    y_train= pickle.load(f)
with open('/content/drive/MyDrive/picklefile/testx.pickle','rb') as f:
    x_test= pickle.load(f)
with open('/content/drive/MyDrive/picklefile/testy.pickle','rb') as f:
    y_test= pickle.load(f)


In [12]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8635, 224, 224, 3), (8635,), (962, 224, 224, 3), (962,))

In [13]:
!pip install -U keras-efficientnet-v2
# Or
!pip install -U git+https://github.com/leondgarse/keras_efficientnet_v2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/leondgarse/keras_efficientnet_v2 to /tmp/pip-req-build-vsbjssn9
  Running command git clone -q https://github.com/leondgarse/keras_efficientnet_v2 /tmp/pip-req-build-vsbjssn9


In [14]:
import tensorflow as tf


In [27]:
#TPU 설정
# TPU gRPC 접근 URI
TPU_PATH = f"grpc://{os.environ['COLAB_TPU_ADDR']}"
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_PATH)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [28]:
strategy = tf.distribute.TPUStrategy(resolver)

In [23]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from tqdm.notebook import tqdm
import pickle
import platform
from tensorflow.keras.applications import EfficientNetB0, efficientnet_v2
import keras_efficientnet_v2
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization
import matplotlib.pyplot as plt
from keras import backend
from keras import layers
from keras.applications import imagenet_utils
from keras.engine import training
from keras.utils import data_utils
from keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.utils import get_custom_objects
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, Activation

In [ ]:
# 라벨 설정
class_names = ['0plus', '1plus', '2plus']

In [ ]:
model = efficientnet_v2.EfficientNetV2S(include_top=False,input_tensor=None,pooling='avg',input_shape=(224, 224, 3), weights = 'imagenet', classes=3, classifier_activation='softmax')

82420632/82420632 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = 'accuracy')

In [ ]:
# test_images = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(test_images[:1000])
# test_images = tf.keras.applications.resnet50.preprocess_input(test_images)
resize = lambda x: tf.image.resize(x, (224, 224)) #람다 함수 : 간단하게 정의 하는 함수 함수명 = 람다 대입연산자 : 내용 출력
test_images = resize(x_test)  # 입력 : x 출력 : tf.image.resize(x, (224, 224))
test_images.shape #reshape 는 행렬의 크기를 바꿀순 없다 10x10 을 바꾸려면 100x1 크기가 유지되는 선에서 쉐입만 바꾸는게 리쉐입이다.

TensorShape([962, 224, 224, 3])

In [ ]:
x_test.shape, y_test.shape

((962, 224, 224, 3), (962,))

In [ ]:
#모델평가
model.evaluate(x_test, y_test)


31/31 [==============================] - 14s 273ms/step - loss: 11.9146 - accuracy: 0.0000e+00


[11.914566993713379, 0.0]

In [ ]:
x_train.shape,y_train.shape

((8635, 224, 224, 3), (8635,))

전체 재학습

In [24]:
#활성화 함수 mish 
from tensorflow.keras.layers import Input, Dense, Flatten, Activation
class Mish(Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'
def mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': Mish(mish)})

In [ ]:

from keras import backend
from keras import layers
from keras.applications import imagenet_utils
from keras.engine import training
from keras.utils import data_utils
from keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras import layers
from tensorflow.keras import losses

In [16]:
base_model = efficientnet_v2.EfficientNetV2L(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')

473176280/473176280 [==============================] - 3s 0us/step


In [ ]:
base_model.trainable = True	#전부돌리면 된다. 트루로 주고			

for layer in base_model.layers[:]:	
  print(layer.name, layer.trainable)

In [ ]:
x_train.shape, y_train.shape

((8635, 224, 224, 3), (8635,))

In [33]:
def create_model():
  base_model = efficientnet_v2.EfficientNetV2L(include_top=False,input_shape=(224, 224, 3), weights = 'imagenet')
  base_model.trainable = True
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = base_model(inputs, training=False)
  x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
  x = tf.keras.layers.Dense(256, activation='mish')(x)
  x= tf.keras.layers.Dropout(0.7)(x)
  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.Model(inputs, outputs)

In [34]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
                      loss = 'sparse_categorical_crossentropy',
                      metrics=['accuracy'])
  model.trainable
  model.fit(x_train, y_train, epochs = 100, validation_data=(x_test, y_test), batch_size= 128)


AttributeError: ignored

tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', -> 성능 판단 지표  
    min_delta=0, -> 개선이라 판단할 최소값    
    patience=0, -> 개선되지않을때 몇 epoch기다렸다 stop할것인지   
    restore_best_weights=False -> 최적weight값으로 복원할지  
    ) 

In [ ]:
#early stoping
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test), batch_size= 128)
model.evaluate()

Epoch 1/40
68/68 [==============================] - 1054s 15s/step - loss: 1.0990 - accuracy: 0.3794 - val_loss: 1.0919 - val_accuracy: 0.3659
Epoch 2/40
68/68 [==============================] - 1036s 15s/step - loss: 1.0590 - accuracy: 0.4307 - val_loss: 1.0976 - val_accuracy: 0.3857
Epoch 3/40
68/68 [==============================] - 1037s 15s/step - loss: 0.9894 - accuracy: 0.5054 - val_loss: 1.1271 - val_accuracy: 0.3971
Epoch 4/40
68/68 [==============================] - 1044s 15s/step - loss: 0.8493 - accuracy: 0.6064 - val_loss: 1.2914 - val_accuracy: 0.3815
Epoch 5/40
68/68 [==============================] - 986s 15s/step - loss: 0.6676 - accuracy: 0.7098 - val_loss: 1.5505 - val_accuracy: 0.3701
Epoch 6/40
68/68 [==============================] - 981s 14s/step - loss: 0.4915 - accuracy: 0.7920 - val_loss: 1.7732 - val_accuracy: 0.3763
Epoch 7/40
68/68 [==============================] - 994s 15s/step - loss: 0.3608 - accuracy: 0.8534 - val_loss: 2.3268 - val_accuracy: 0.3586
Ep

In [ ]:
## 일부 재학습 모델
base_model = efficientnet_v2.EfficientNetV2S(include_top=False,input_tensor=None,pooling='avg',input_shape=(224, 224, 3), weights = None, classes=3)

# base_model.trainable = False

# for layer in base_model.layers[-10:]: 
#     layer.trainable = True				

base_model.trainable = True

for layer in base_model.layers[:-10]: 
    layer.trainable = False				


inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False) # batchnorm 부분 update 방지

x = tf.keras.layers.Flatten(input_shape=model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='leaky_relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model1 = tf.keras.Model(inputs, outputs)
with strategy.scope():
  
  model1.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  model1.fit(x_train, y_train, epochs = 5, validation_data=(x_test, y_test), 
                   batch_size= 125)

In [ ]:
model1.fit(x_train, y_train, epochs = 5, validation_data=(x_test, y_test), 
                   batch_size= 125)

Epoch 1/5
70/70 [==============================] - 84s 1s/step - loss: 1.6845 - accuracy: 0.3455 - val_loss: 1.1324 - val_accuracy: 0.3565
Epoch 2/5
70/70 [==============================] - 68s 980ms/step - loss: 1.1774 - accuracy: 0.3439 - val_loss: 1.1162 - val_accuracy: 0.3534
Epoch 3/5
70/70 [==============================] - 69s 984ms/step - loss: 1.1533 - accuracy: 0.3531 - val_loss: 1.1088 - val_accuracy: 0.3534
Epoch 4/5
70/70 [==============================] - 69s 982ms/step - loss: 1.1435 - accuracy: 0.3501 - val_loss: 1.1073 - val_accuracy: 0.3565
Epoch 5/5
70/70 [==============================] - 69s 982ms/step - loss: 1.1329 - accuracy: 0.3508 - val_loss: 1.1037 - val_accuracy: 0.3534
